In [1]:
import cv2
import numpy as np
import keras
import matplotlib.pyplot as plt
import os
import glob

In [2]:
model = keras.models.load_model("conv_autoencoder_model_v4.h5", compile = False)
model.compile()

In [13]:
def detect(save=False , detection_folder = "",detected_folder = "" , file_name="", threshold = 0.0019):
    if save:
        file_out = detected_folder + "/" + file_name
    file_in = detection_folder + "/" + file_name
    x, y = 0, 0
    img = cv2.imread(file_in)
    clear = cv2.fastNlMeansDenoisingColored(img,None,10,10,7,21)

    width = len(img[0])
    height = len(img)
    while x + 32 < width:
        while y + 32 < height:
            img_part = clear[y:y+32, x:x+32]
            recon_part = model(img_part[np.newaxis, :,  :, :])
            recon_part = np.array(recon_part)
            for i in range(32):
                for j in range(32):
                    if (np.sum(np.square(img_part[i][j]-recon_part[0][i][j])/(3*255*255))) > threshold:
                        img[y+i][x+j] = [0, 0, 255]

            y = y + 32
        x = x + 32
        y = 0
    
    #cv2.imwrite(file_out, img)
    """
    #cv2.imshow("reconstruted", img) show image
    k = cv2.waitKey(0)
    if k == 27:         # wait for ESC key to exit
        cv2.destroyAllWindows()
    elif k == ord('s') and save:  # wait for 's' key to save and exit
        
        cv2.destroyAllWindows()
    #return(img)
    """

In [14]:
def detect_folder(folder,detected_folder):
    T = 0.001
    folder_path = os.path.join(folder, '*.*')
    list_images = glob.glob(folder_path)
    for file in list_images:
        img_name = os.path.basename(file)
        if not os.path.exists(detected_folder + "/" + img_name):
            print(img_name)
            detect(save=True , detection_folder = folder, detected_folder = detected_folder , file_name = img_name , threshold = T)
        
    

In [15]:
detect_folder("images_for_detection","images_detected")